In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install -q datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# Dataset

Load the dataset created by Niels Rogge [here](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Donut/RVL-CDIP/Preparing_an_image_classification_dataset_for_Donut.ipynb). This is a small subset of the entire  RVL-CDIP.

In [4]:
from datasets import load_dataset

dataset = load_dataset("nielsr/rvl_cdip_10_examples_per_class_donut")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/160 [00:00<?, ? examples/s]

In [5]:
id2label = {id: label for id, label in enumerate(dataset['train'].features['label'].names)}
print(id2label)

{0: 'letter', 1: 'form', 2: 'email', 3: 'handwritten', 4: 'advertisement', 5: 'scientific report', 6: 'scientific publication', 7: 'specification', 8: 'file folder', 9: 'news article', 10: 'budget', 11: 'invoice', 12: 'presentation', 13: 'questionnaire', 14: 'resume', 15: 'memo'}


# Model

Here is the most important part, we created the model for the encoder only donut with classification head in the end. Basically, the encoder will work as feature extraction step. Its pooled output will pass through a dropout and a classifier layer, which maps from number of features to number of classes.

In [6]:
from transformers import DonutSwinModel, DonutSwinPreTrainedModel
from torch import nn
import torch

class DonutForImageClassification(DonutSwinPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.swin = DonutSwinModel(config)
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(self.swin.num_features, config.num_labels)

    def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
        outputs = self.swin(pixel_values)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [20]:
from transformers import VisionEncoderDecoderConfig, DonutProcessor, VisionEncoderDecoderModel
image_size = [1280, 960]
config = VisionEncoderDecoderConfig.from_pretrained("nielsr/donut-base")
config.encoder.image_size = image_size

processor = DonutProcessor.from_pretrained("nielsr/donut-base")
donut_model = VisionEncoderDecoderModel.from_pretrained("nielsr/donut-base", config=config)

processor.feature_extractor.size = image_size[::-1] # should be (width, height)
processor.feature_extractor.do_align_long_axis = False

donut_model.encoder.save_pretrained("donut_encoder")

In [21]:
model = DonutForImageClassification.from_pretrained("donut_encoder", num_labels=16)

Some weights of DonutForImageClassification were not initialized from the model checkpoint at donut_encoder and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now we can prepare the dataset to be loaded using PyTorch

In [22]:
import os
from PIL import Image
from torch.utils.data import Dataset

class DocumentDataset(Dataset):
    def __init__(self, dataset_name_or_path, split):
        self.split = split
        self.dataset = load_dataset(dataset_name_or_path, split=self.split)
        print(self.dataset)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        pixel_values = processor(sample["image"].convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()
        label = sample['label']

        encoding = dict(pixel_values=pixel_values,
                        label=label)
        return encoding

In [23]:
train_dataset = DocumentDataset("nielsr/rvl_cdip_10_examples_per_class_donut", split="train")

Dataset({
    features: ['image', 'label', 'ground_truth'],
    num_rows: 160
})


In [24]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [25]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['pixel_values', 'label'])


# Training

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

DonutForImageClassification(
  (swin): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
            

To be able to benchmark it later, we saved the model after 5, 10, 15, 20 epochs trained. In this way, we have more detail on the convergence and learning of the model.

In [27]:
import torch
from tqdm.auto import tqdm


optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(20):
    print("Epoch:", epoch+1)
    model.train()
    for i, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        pixel_values = batch["pixel_values"]
        labels = batch["label"]
        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print("Loss:", loss.item())

    if epoch + 1 in {5, 10, 15, 20}:
        checkpoint_path = f"model_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Model saved at {checkpoint_path}")

Epoch: 1


  0%|          | 0/160 [00:00<?, ?it/s]

Loss: 3.70231556892395
Loss: 3.024982452392578
Epoch: 2


  0%|          | 0/160 [00:00<?, ?it/s]

Loss: 2.541522264480591
Loss: 2.860440969467163
Epoch: 3


...

Loss: 1.6614376306533813
Loss: 0.007626701612025499
Epoch: 19


  0%|          | 0/160 [00:00<?, ?it/s]

Loss: 0.0004589696181938052
Loss: 1.743046522140503
Epoch: 20


  0%|          | 0/160 [00:00<?, ?it/s]

Loss: 0.1126399114727974
Loss: 2.5018186569213867
Model saved at model_epoch_20.pth


# Evaluate and Benchmark

Here we simply run the same cell 4 times for each model saved, getting their average inference time and accuracy.

In [28]:
# Evaluation loop
import time
import numpy as np

model.load_state_dict(torch.load('/content/model_epoch_5.pth'))
model.eval()
correct = 0
total = 0
dataset = load_dataset("nielsr/rvl_cdip_10_examples_per_class_donut", split="test")
inference_time = []

with torch.no_grad():
    for sample in dataset:
        pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        labels = sample["label"]
        start_time = time.time()
        outputs = model(pixel_values)
        end_time = time.time()
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels).sum().item()
        inference_time.append(end_time - start_time)


accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')
print(f'Predicted {correct} correctly out of {total}!')
print(f"Average Inference time: {np.mean(inference_time):.6f} seconds")

Accuracy: 31.25%
Predicted 50 correctly out of 160!
Average Inference time: 0.029618 seconds


In [29]:
# Evaluation loop
import time
import numpy as np

model.load_state_dict(torch.load('/content/model_epoch_10.pth'))
model.eval()
correct = 0
total = 0
dataset = load_dataset("nielsr/rvl_cdip_10_examples_per_class_donut", split="test")
inference_time = []

with torch.no_grad():
    for sample in dataset:
        pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        labels = sample["label"]
        start_time = time.time()
        outputs = model(pixel_values)
        end_time = time.time()
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels).sum().item()
        inference_time.append(end_time - start_time)


accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')
print(f'Predicted {correct} correctly out of {total}!')
print(f"Average Inference time: {np.mean(inference_time):.6f} seconds")

Accuracy: 35.0%
Predicted 56 correctly out of 160!
Average Inference time: 0.028845 seconds


In [30]:
# Evaluation loop
import time
import numpy as np

model.load_state_dict(torch.load('/content/model_epoch_15.pth'))
model.eval()
correct = 0
total = 0
dataset = load_dataset("nielsr/rvl_cdip_10_examples_per_class_donut", split="test")
inference_time = []

with torch.no_grad():
    for sample in dataset:
        pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        labels = sample["label"]
        start_time = time.time()
        outputs = model(pixel_values)
        end_time = time.time()
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels).sum().item()
        inference_time.append(end_time - start_time)


accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')
print(f'Predicted {correct} correctly out of {total}!')
print(f"Average Inference time: {np.mean(inference_time):.6f} seconds")

Accuracy: 41.875%
Predicted 67 correctly out of 160!
Average Inference time: 0.028193 seconds


In [31]:
# Evaluation loop
import time
import numpy as np

model.load_state_dict(torch.load('/content/model_epoch_20.pth'))
model.eval()
correct = 0
total = 0
dataset = load_dataset("nielsr/rvl_cdip_10_examples_per_class_donut", split="test")
inference_time = []

with torch.no_grad():
    for sample in dataset:
        pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        labels = sample["label"]
        start_time = time.time()
        outputs = model(pixel_values)
        end_time = time.time()
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels).sum().item()
        inference_time.append(end_time - start_time)


accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')
print(f'Predicted {correct} correctly out of {total}!')
print(f"Average Inference time: {np.mean(inference_time):.6f} seconds")

Accuracy: 51.875%
Predicted 83 correctly out of 160!
Average Inference time: 0.029348 seconds
